In [46]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as stats
import scipy as s
from scipy import stats
# import seaborn as sns
plt.rcParams['font.size'] = 18
#plt.rcParams['font.family'] = "serif"
tdir = 'in'
major = 5.0
minor = 3.0
plt.rcParams['xtick.direction'] = tdir
plt.rcParams['ytick.direction'] = tdir
plt.rcParams['xtick.major.size'] = major
plt.rcParams['xtick.minor.size'] = minor
plt.rcParams['ytick.major.size'] = major
plt.rcParams['ytick.minor.size'] = minor

In [7]:
def get_month_data(data_dir, month, train=True):
    
    if train:
        data_dir = data_dir + str(month) + '/'
        XY_train = np.load(data_dir + 'XY_train.npz')
        X_tr, Y_tr = XY_train['X_train'], XY_train['Y_train']
        
        return np.array(X_tr), np.array(Y_tr)
    else:
        data_dir = data_dir + str(month) + '/'
        XY_test = np.load(data_dir + 'XY_test.npz')
        X_test, Y_test = XY_test['X_test'], XY_test['Y_test']

        return np.array(X_test), np.array(Y_test) 
    
all_task_months = ['2018-01', '2018-02', '2018-03', '2018-04', '2018-05', '2018-06',
                   '2018-07', '2018-08', '2018-09', '2018-10', '2018-11', '2018-12']


cnt_samples = 0
for month in all_task_months:
    pre_X_tr, pre_Y_tr = get_month_data('../../month_based_processing_with_family_labels/', month)
    
    print(f'month {month}\t\tpre_Y_tr {pre_Y_tr.shape}')
    cnt_samples += len(pre_Y_tr)
print(f'total samples {cnt_samples}')

month 2018-01		pre_Y_tr (55722,)
month 2018-02		pre_Y_tr (48723,)
month 2018-03		pre_Y_tr (37372,)
month 2018-04		pre_Y_tr (46873,)
month 2018-05		pre_Y_tr (41320,)
month 2018-06		pre_Y_tr (43560,)
month 2018-07		pre_Y_tr (46278,)
month 2018-08		pre_Y_tr (40882,)
month 2018-09		pre_Y_tr (56492,)
month 2018-10		pre_Y_tr (77772,)
month 2018-11		pre_Y_tr (90000,)
month 2018-12		pre_Y_tr (90000,)
total samples 674994


In [64]:
def get_average_all_exps(result_dict, get_exps_min=False):
    
    multiexps = {}
    for k, runs in result_dict.items():
        #print(k)
        #for runs in precs_dict[k]:
        #print(runs)
        for ind, run in enumerate(runs):
            #print(run)
            try:
                if ind in multiexps.keys():
                    multiexps[ind].append(run)
                else:
                    multiexps[ind] = [run]
            except: continue


    exps_mean = []
    exps_min = []
    for exp in multiexps.keys():
        exps_mean.append(np.mean(multiexps[exp]))
        exps_min.append(np.min(multiexps[exp]))
        #print(np.mean(multiexps[exp]), np.std(multiexps[exp]))

    assert len(exps_mean) == len(exps_min)

    #print(f'Mean {np.mean(exps_mean)* 100:.2f}\u00B1{np.std(exps_mean)* 100:.2f}')
    #print(f'Min {np.mean(exps_min)* 100:.2f}\u00B1{np.std(exps_min)* 100:.2f}')
    
    if get_exps_min:
        return exps_mean, exps_min
    else:
        return exps_mean 
    
    
    
def V2_ifs_results(scenario, replay_config, ifs_option='ratio', joint=False,
                cnt_rate=0.1, anomaly_perct=0.5, memory_budget=1000):

    target_classes = 100
    
    results_dir = './Submission_Domain/'

    if replay_config == 'grs':
        if joint:
            result_file = str(replay_config)  + '_joint_results.txt'
        else:
            result_file = str(replay_config)  + '_' +\
                    str(memory_budget)  + '_results.txt'
    elif replay_config == 'ifs':
        result_file = 'ifs_' + str(ifs_option) + '_' + str(memory_budget) + '_results.txt'
    else:
        result_file = str(replay_config)  + '_' +\
                    str(memory_budget)  + '_results.txt'

        
    result_path = results_dir + result_file 
    print(result_path)
    
    accs_dict = {}
    precs_dict = {}
    recalls_dict = {}
    f1_dict = {}

    with open(result_path) as f:
            lines = f.readlines()
            #print(lines)
            for line in lines:
                #print(line)
                infos = line.split('\t')
                task = infos[0]
                #print(int(task))
                acc, prec, recal, f1 = float(infos[1]), float(infos[2]), float(infos[3]), float(infos[4])

                if task in accs_dict.keys():
                    accs_dict[task].append(acc)
                    precs_dict[task].append(prec)
                    recalls_dict[task].append(recal)
                    f1_dict[task].append(f1)
                else:
                    accs_dict[task] = [acc]
                    precs_dict[task] = [prec]
                    recalls_dict[task] = [recal]
                    f1_dict[task] = [f1]

    #print(accs_dict)
    
    #accs_all_task = []
    accs_all = get_average_all_exps(accs_dict, get_exps_min=False)
    precs_all, precs_all_min = get_average_all_exps(precs_dict, get_exps_min=True)
    
    recalls_all =get_average_all_exps(recalls_dict, get_exps_min=False)
    f1_all = get_average_all_exps(f1_dict, get_exps_min=False)
    
    
    precs_all_task = []
    f1_all_task = []
    for k, v in precs_dict.items():
        precs_all_task.append(v)
        f1_all_task.append(f1_dict[k])
        

    #print()
    if replay_config == 'grs':
        if joint:
            print(f'\t\t {str(replay_config)}-Joint \n#################\t###################\t###################\t###################\nprec   {np.mean(precs_all)* 100:.2f}\u00B1{np.std(precs_all) * 100:.2f} \tprec-min\t{np.mean(precs_all_min)* 100:.2f}\u00B1{np.std(precs_all_min) * 100:.2f} \trecall\t {np.mean(recalls_all)* 100:.2f}\u00B1{np.std(recalls_all) * 100:.2f} \tf1\t{np.mean(f1_all)* 100:.2f}\u00B1{np.std(f1_all) * 100:.2f} \n#################\t###################\t###################\t###################')
        else:
            print(f'\t\t {str(replay_config)}-{str(memory_budget)}\n#################\t###################\t###################\t###################\nprec   {np.mean(precs_all)* 100:.2f}\u00B1{np.std(precs_all) * 100:.2f} \tprec-min\t{np.mean(precs_all_min)* 100:.2f}\u00B1{np.std(precs_all_min) * 100:.2f} \trecall\t {np.mean(recalls_all)* 100:.2f}\u00B1{np.std(recalls_all) * 100:.2f} \tf1\t{np.mean(f1_all)* 100:.2f}\u00B1{np.std(f1_all) * 100:.2f} \n#################\t###################\t###################\t###################')
    if replay_config == 'ifs':
        print(f'\t\t {str(replay_config)}-{str(ifs_option)}-{str(memory_budget)}\n#################\t###################\t###################\t###################\nprec   {np.mean(precs_all)* 100:.2f}\u00B1{np.std(precs_all) * 100:.2f} \tprec-min\t{np.mean(precs_all_min)* 100:.2f}\u00B1{np.std(precs_all_min) * 100:.2f} \trecall\t {np.mean(recalls_all)* 100:.2f}\u00B1{np.std(recalls_all) * 100:.2f} \tf1\t{np.mean(f1_all)* 100:.2f}\u00B1{np.std(f1_all) * 100:.2f} \n#################\t###################\t###################\t###################')
    
    print()
    return  np.array(precs_all_task), np.array(accs_all), np.array(precs_all), np.array(recalls_all), np.array(f1_all_task), np.array(f1_all)


def ifs_results(scenario, replay_config, ifs_option='ratio', joint=False,
                cnt_rate=0.1, anomaly_perct=0.5, min_samples=1, goodware_ifs=False, memory_budget=1000):

    #target_classes = 100
    
    results_dir = './Submission_Domain/'

    if replay_config == 'grs':
        if joint:
            result_file = str(replay_config)  + '_joint_results.txt'
        else:
            result_file = str(replay_config)  + '_' +\
                    str(memory_budget)  + '_results.txt'
            
    elif replay_config == 'ifs':
        if goodware_ifs:
            if min_samples > 1:
                result_file = 'ifs_good_' + str(ifs_option) + '_' + str(min_samples) + '_' + str(memory_budget) + '_results.txt'
            else:
                result_file = 'ifs_good_' + str(ifs_option) + '_' + str(memory_budget) + '_results.txt'
        else:
            if min_samples > 1:
                result_file = 'ifs_' + str(ifs_option) + '_' + str(min_samples) + '_' + str(memory_budget) + '_results.txt'
            else:
                result_file = 'ifs_' + str(ifs_option) + '_' + str(memory_budget) + '_results.txt'

    elif replay_config == 'aws':
        if goodware_ifs:
            if min_samples > 1:
                result_file = 'aws_good_' + str(ifs_option) + '_' + str(min_samples) + '_' + str(memory_budget) + '_results.txt'
            else:
                result_file = 'aws_good_' + str(ifs_option) + '_' + str(memory_budget) + '_results.txt'
        else:
            if min_samples > 1:
                result_file = 'aws_' + str(ifs_option) + '_' + str(min_samples) + '_' + str(memory_budget) + '_results.txt'
            else:
                result_file = 'aws_' + str(ifs_option) + '_' + str(memory_budget) + '_results.txt'

        
    else:
        result_file = str(replay_config)  + '_' +\
                    str(memory_budget)  + '_results.txt'

        
    result_path = results_dir + result_file 
    #print(result_path)
    
    accs_dict = {}
    precs_dict = {}
    recalls_dict = {}
    f1_dict = {}

    with open(result_path) as f:
            lines = f.readlines()
            #print(lines)
            for line in lines:
                #print(line)
                infos = line.split('\t')
                #print(infos)
                task = infos[0]
                #print(int(task))
                acc, prec, recal, f1 = float(infos[1]), float(infos[2]), float(infos[3]), float(infos[4])

                if task in accs_dict.keys():
                    accs_dict[task].append(acc)
                    precs_dict[task].append(prec)
                    recalls_dict[task].append(recal)
                    f1_dict[task].append(f1)
                else:
                    accs_dict[task] = [acc]
                    precs_dict[task] = [prec]
                    recalls_dict[task] = [recal]
                    f1_dict[task] = [f1]

    #print(accs_dict)
    
    #accs_all_task = []
    accs_all = get_average_all_exps(accs_dict, get_exps_min=False)
    precs_all, precs_all_min = get_average_all_exps(precs_dict, get_exps_min=True)
    
    recalls_all =get_average_all_exps(recalls_dict, get_exps_min=False)
    f1_all = get_average_all_exps(f1_dict, get_exps_min=False)
    
    
    precs_all_task = []
    f1_all_task = []
    for k, v in precs_dict.items():
        precs_all_task.append(v)
        f1_all_task.append(f1_dict[k])
        

    #print()
    if replay_config == 'grs':
        if joint:
            #print(f'\t\t {str(replay_config)}-Joint \n#################\t###################\t###################\t###################\nprec   {np.mean(precs_all)* 100:.2f}\u00B1{np.std(precs_all) * 100:.2f} \tprec-min\t{np.mean(precs_all_min)* 100:.2f}\u00B1{np.std(precs_all_min) * 100:.2f} \trecall\t {np.mean(recalls_all)* 100:.2f}\u00B1{np.std(recalls_all) * 100:.2f} \tf1\t{np.mean(f1_all)* 100:.2f}\u00B1{np.std(f1_all) * 100:.2f} \n#################\t###################\t###################\t###################')
            print(f'\t {str(replay_config)}-Joint\tprec-min\t{np.mean(precs_all_min)* 100:.1f}$\pm${np.std(precs_all_min) * 100:.1f}')
        else:
            print(f'\t {str(replay_config)}-{str(memory_budget)}\tprec-min\t{np.mean(precs_all_min)* 100:.1f}$\pm${np.std(precs_all_min) * 100:.1f}')
            #print(f'\t\t {str(replay_config)}-{str(memory_budget)}\n#################\t###################\t###################\t###################\nprec   {np.mean(precs_all)* 100:.2f}\u00B1{np.std(precs_all) * 100:.2f} \tprec-min\t{np.mean(precs_all_min)* 100:.2f}\u00B1{np.std(precs_all_min) * 100:.2f} \trecall\t {np.mean(recalls_all)* 100:.2f}\u00B1{np.std(recalls_all) * 100:.2f} \tf1\t{np.mean(f1_all)* 100:.2f}\u00B1{np.std(f1_all) * 100:.2f} \n#################\t###################\t###################\t###################')
    
    if replay_config == 'ifs':
        if goodware_ifs:
            if min_samples > 1:
                print(f'\tGood-{str(replay_config)}-{str(ifs_option)}-{str(memory_budget)}-{str(min_samples)}\tprec\t{np.mean(precs_all)* 100:.1f}$\pm${np.std(precs_all) * 100:.1f}\tprec-min\t{np.mean(precs_all_min)* 100:.1f}$\pm${np.std(precs_all_min) * 100:.1f}\n')
            else:
                #print(f'\t\t {str(replay_config)}-{str(ifs_option)}-{str(memory_budget)}\n#################\t###################\t###################\t###################\nprec   {np.mean(precs_all)* 100:.2f}\u00B1{np.std(precs_all) * 100:.2f} \tprec-min\t{np.mean(precs_all_min)* 100:.2f}\u00B1{np.std(precs_all_min) * 100:.2f} \trecall\t {np.mean(recalls_all)* 100:.2f}\u00B1{np.std(recalls_all) * 100:.2f} \tf1\t{np.mean(f1_all)* 100:.2f}\u00B1{np.std(f1_all) * 100:.2f} \n#################\t###################\t###################\t###################')
                print(f'\tGood-{str(replay_config)}-{str(ifs_option)}-{str(memory_budget)}\tprec\t{np.mean(precs_all)* 100:.1f}$\pm${np.std(precs_all) * 100:.1f}\tprec-min\t{np.mean(precs_all_min)* 100:.1f}$\pm${np.std(precs_all_min) * 100:.1f}\n')
                #print(f'\t{str(replay_config)}-{str(ifs_option)}-{str(memory_budget)}\tprec-min\t{np.mean(precs_all_min)* 100:.1f}$\pm${np.std(precs_all_min) * 100:.1f}****\n')
        else:
            if min_samples > 1:
                print(f'\t{str(replay_config)}-{str(ifs_option)}-{str(memory_budget)}-{str(min_samples)}\tprec\t{np.mean(precs_all)* 100:.1f}$\pm${np.std(precs_all) * 100:.1f}\tprec-min\t{np.mean(precs_all_min)* 100:.1f}$\pm${np.std(precs_all_min) * 100:.1f}\n')
            else:
                #print(f'\t\t {str(replay_config)}-{str(ifs_option)}-{str(memory_budget)}\n#################\t###################\t###################\t###################\nprec   {np.mean(precs_all)* 100:.2f}\u00B1{np.std(precs_all) * 100:.2f} \tprec-min\t{np.mean(precs_all_min)* 100:.2f}\u00B1{np.std(precs_all_min) * 100:.2f} \trecall\t {np.mean(recalls_all)* 100:.2f}\u00B1{np.std(recalls_all) * 100:.2f} \tf1\t{np.mean(f1_all)* 100:.2f}\u00B1{np.std(f1_all) * 100:.2f} \n#################\t###################\t###################\t###################')
                print(f'\t{str(replay_config)}-{str(ifs_option)}-{str(memory_budget)}\tprec\t{np.mean(precs_all)* 100:.1f}$\pm${np.std(precs_all) * 100:.1f}\tprec-min\t{np.mean(precs_all_min)* 100:.1f}$\pm${np.std(precs_all_min) * 100:.1f}\n')
                #print(f'\t{str(replay_config)}-{str(ifs_option)}-{str(memory_budget)}\tprec-min\t{np.mean(precs_all_min)* 100:.1f}$\pm${np.std(precs_all_min) * 100:.1f}****\n')
    if replay_config == 'aws':
        if goodware_ifs:
            if min_samples > 1:
                print(f'\tGood-{str(replay_config)}-{str(ifs_option)}-{str(memory_budget)}-{str(min_samples)}\tprec\t{np.mean(precs_all)* 100:.1f}$\pm${np.std(precs_all) * 100:.1f}\tprec-min\t{np.mean(precs_all_min)* 100:.1f}$\pm${np.std(precs_all_min) * 100:.1f}\n')
            else:
                #print(f'\t\t {str(replay_config)}-{str(ifs_option)}-{str(memory_budget)}\n#################\t###################\t###################\t###################\nprec   {np.mean(precs_all)* 100:.2f}\u00B1{np.std(precs_all) * 100:.2f} \tprec-min\t{np.mean(precs_all_min)* 100:.2f}\u00B1{np.std(precs_all_min) * 100:.2f} \trecall\t {np.mean(recalls_all)* 100:.2f}\u00B1{np.std(recalls_all) * 100:.2f} \tf1\t{np.mean(f1_all)* 100:.2f}\u00B1{np.std(f1_all) * 100:.2f} \n#################\t###################\t###################\t###################')
                print(f'\tGood-{str(replay_config)}-{str(ifs_option)}-{str(memory_budget)}\tprec\t{np.mean(precs_all)* 100:.1f}$\pm${np.std(precs_all) * 100:.1f}\tprec-min\t{np.mean(precs_all_min)* 100:.1f}$\pm${np.std(precs_all_min) * 100:.1f}\n')
                #print(f'\t{str(replay_config)}-{str(ifs_option)}-{str(memory_budget)}\tprec-min\t{np.mean(precs_all_min)* 100:.1f}$\pm${np.std(precs_all_min) * 100:.1f}****\n')
        else:
            if min_samples > 1:
                print(f'\t{str(replay_config)}-{str(ifs_option)}-{str(memory_budget)}-{str(min_samples)}\tprec\t{np.mean(precs_all)* 100:.1f}$\pm${np.std(precs_all) * 100:.1f}\tprec-min\t{np.mean(precs_all_min)* 100:.1f}$\pm${np.std(precs_all_min) * 100:.1f}\n')
            else:
                #print(f'\t\t {str(replay_config)}-{str(ifs_option)}-{str(memory_budget)}\n#################\t###################\t###################\t###################\nprec   {np.mean(precs_all)* 100:.2f}\u00B1{np.std(precs_all) * 100:.2f} \tprec-min\t{np.mean(precs_all_min)* 100:.2f}\u00B1{np.std(precs_all_min) * 100:.2f} \trecall\t {np.mean(recalls_all)* 100:.2f}\u00B1{np.std(recalls_all) * 100:.2f} \tf1\t{np.mean(f1_all)* 100:.2f}\u00B1{np.std(f1_all) * 100:.2f} \n#################\t###################\t###################\t###################')
                print(f'\t{str(replay_config)}-{str(ifs_option)}-{str(memory_budget)}\tprec\t{np.mean(precs_all)* 100:.1f}$\pm${np.std(precs_all) * 100:.1f}\tprec-min\t{np.mean(precs_all_min)* 100:.1f}$\pm${np.std(precs_all_min) * 100:.1f}\n')
                #print(f'\t{str(replay_config)}-{str(ifs_option)}-{str(memory_budget)}\tprec-min\t{np.mean(precs_all_min)* 100:.1f}$\pm${np.std(precs_all_min) * 100:.1f}****\n')
    #print()
    
    
    return  np.array(precs_all_task), np.array(accs_all), np.array(precs_all), np.array(recalls_all), np.array(f1_all_task), np.array(f1_all)




# scenario, replay_config, joint=False,
#                 cnt_rate=0.1, anomaly_perct=0.5, memory_budget=1000

# grs_joint, _, _, _, _, _ = ifs_results('domain', replay_config='grs', joint=True) 


# grs_1K, _, _, _, _, _ = ifs_results('domain', replay_config='grs', memory_budget=1000)
# grs_10K, _, _, _, _, _ = ifs_results('domain', replay_config='grs', memory_budget=10000)
# grs_50K, _, _, _, _, _ = ifs_results('domain', replay_config='grs', memory_budget=50000)
# grs_100K, _, _, _, _, _ = ifs_results('domain', replay_config='grs', memory_budget=100000)
# grs_150K, _, _, _, _, _ = ifs_results('domain', replay_config='grs', memory_budget=150000)
# grs_200K, _, _, _, _, _ = ifs_results('domain', replay_config='grs', memory_budget=200000)
# grs_250K, _, _, _, _, _ = ifs_results('domain', replay_config='grs', memory_budget=250000)
# grs_300K, _, _, _, _, _ = ifs_results('domain', replay_config='grs', memory_budget=300000)
# grs_350K, _, _, _, _, _ = ifs_results('domain', replay_config='grs', memory_budget=350000)
# grs_400K, _, _, _, _, _ = ifs_results('domain', replay_config='grs', memory_budget=400000);

In [49]:
ifs_1K, _, _, _, _, _ = ifs_results('domain',  replay_config='ifs', ifs_option='ratio', memory_budget=1000)
ifs_10K, _, _, _, _, _ = ifs_results('domain',  replay_config='ifs', ifs_option='ratio', memory_budget=10000)
ifs_50K, _, _, _, _, _ = ifs_results('domain',  replay_config='ifs', ifs_option='ratio', memory_budget=50000)
ifs_100K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='ratio', memory_budget=100000)
ifs_150K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='ratio', memory_budget=150000)
ifs_200K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='ratio', memory_budget=200000)
ifs_250K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='ratio', memory_budget=250000)
ifs_300K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='ratio', memory_budget=300000)
ifs_350K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='ratio', memory_budget=350000)
ifs_400K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='ratio', memory_budget=400000);

	ifs-ratio-1000	prec	93.7$\pm$0.1	prec-min	92.6$\pm$0.1

	ifs-ratio-10000	prec	94.7$\pm$0.1	prec-min	94.1$\pm$0.1

	ifs-ratio-50000	prec	95.4$\pm$0.1	prec-min	94.6$\pm$0.3

	ifs-ratio-100000	prec	95.8$\pm$0.0	prec-min	94.3$\pm$0.0

	ifs-ratio-150000	prec	96.1$\pm$0.0	prec-min	94.8$\pm$0.4

	ifs-ratio-200000	prec	96.0$\pm$0.1	prec-min	94.3$\pm$0.1

	ifs-ratio-250000	prec	96.1$\pm$0.0	prec-min	94.4$\pm$0.5

	ifs-ratio-300000	prec	96.1$\pm$0.0	prec-min	94.7$\pm$0.1

	ifs-ratio-350000	prec	96.3$\pm$0.0	prec-min	95.3$\pm$0.1

	ifs-ratio-400000	prec	96.1$\pm$0.0	prec-min	94.0$\pm$0.0



In [47]:
ifs_1K, _, _, _, _, _ = ifs_results('domain',  replay_config='ifs', ifs_option='uniform', memory_budget=1000)
ifs_10K, _, _, _, _, _ = ifs_results('domain',  replay_config='ifs', ifs_option='uniform', memory_budget=10000)
ifs_50K, _, _, _, _, _ = ifs_results('domain',  replay_config='ifs', ifs_option='uniform', memory_budget=50000)
ifs_100K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='uniform', memory_budget=100000)
ifs_150K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='uniform', memory_budget=150000)
ifs_200K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='uniform', memory_budget=200000)
ifs_250K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='uniform', memory_budget=250000)
ifs_300K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='uniform', memory_budget=300000)
ifs_350K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='uniform', memory_budget=350000)
ifs_400K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='uniform', memory_budget=400000);

	ifs-uniform-1000	prec	93.6$\pm$0.2	prec-min	92.5$\pm$0.2

	ifs-uniform-10000	prec	94.0$\pm$0.2	prec-min	93.0$\pm$0.2

	ifs-uniform-50000	prec	95.1$\pm$0.1	prec-min	94.4$\pm$0.2

	ifs-uniform-100000	prec	95.3$\pm$0.1	prec-min	94.3$\pm$0.5

	ifs-uniform-150000	prec	95.4$\pm$0.1	prec-min	94.4$\pm$0.6

	ifs-uniform-200000	prec	95.5$\pm$0.1	prec-min	94.5$\pm$0.3

	ifs-uniform-250000	prec	95.6$\pm$0.1	prec-min	94.3$\pm$0.5

	ifs-uniform-300000	prec	95.7$\pm$0.1	prec-min	94.2$\pm$0.2

	ifs-uniform-350000	prec	95.4$\pm$0.2	prec-min	93.3$\pm$1.9

	ifs-uniform-400000	prec	95.7$\pm$0.1	prec-min	94.2$\pm$0.4



/home/mr6564/anaconda3/envs/PyTorch/lib/python3.7/site-packages/ipykernel_launcher.py:222: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [48]:
ifs_1K, _, _, _, _, _ = ifs_results('domain',  replay_config='ifs', ifs_option='gifs', memory_budget=1000)
ifs_10K, _, _, _, _, _ = ifs_results('domain',  replay_config='ifs', ifs_option='gifs', memory_budget=10000)
ifs_50K, _, _, _, _, _ = ifs_results('domain',  replay_config='ifs', ifs_option='gifs', memory_budget=50000)
ifs_100K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='gifs', memory_budget=100000)
ifs_150K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='gifs', memory_budget=150000)
ifs_200K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='gifs', memory_budget=200000)
ifs_250K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='gifs', memory_budget=250000)
ifs_300K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='gifs', memory_budget=300000)
ifs_350K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='gifs', memory_budget=350000)
ifs_400K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='gifs', memory_budget=400000);

	ifs-gifs-1000	prec	93.6$\pm$0.1	prec-min	91.8$\pm$0.3

	ifs-gifs-10000	prec	94.5$\pm$0.1	prec-min	93.7$\pm$0.1

	ifs-gifs-50000	prec	95.2$\pm$0.0	prec-min	93.9$\pm$0.3

	ifs-gifs-100000	prec	94.5$\pm$1.1	prec-min	93.9$\pm$0.5

	ifs-gifs-150000	prec	94.9$\pm$0.8	prec-min	94.2$\pm$0.4

	ifs-gifs-200000	prec	94.8$\pm$0.9	prec-min	94.2$\pm$0.5

	ifs-gifs-250000	prec	94.9$\pm$0.9	prec-min	93.3$\pm$1.0

	ifs-gifs-300000	prec	95.5$\pm$0.8	prec-min	94.4$\pm$0.6

	ifs-gifs-350000	prec	95.7$\pm$0.6	prec-min	94.5$\pm$0.4

	ifs-gifs-400000	prec	95.0$\pm$1.3	prec-min	93.8$\pm$0.7



/home/mr6564/anaconda3/envs/PyTorch/lib/python3.7/site-packages/ipykernel_launcher.py:222: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [22]:
ifsM_100_2, _, _, _, _, _ = ifs_results('domain', replay_config='ifs',\
                          ifs_option='mix', min_samples=2, memory_budget=1000)
ifsM_100_5, _, _, _, _, _ = ifs_results('domain', replay_config='ifs',\
                          ifs_option='mix', min_samples=5, memory_budget=1000)
ifsM_100_10, _, _, _, _, _ = ifs_results('domain', replay_config='ifs',\
                          ifs_option='mix', min_samples=10, memory_budget=1000)
ifsM_100_15, _, _, _, _, _ = ifs_results('domain', replay_config='ifs',\
                          ifs_option='mix', min_samples=15, memory_budget=1000)
ifsM_100_20, _, _, _, _, _ = ifs_results('domain', replay_config='ifs',\
                          ifs_option='mix', min_samples=20, memory_budget=1000)


	ifs-mix-1000-2	prec	93.6$\pm$0.2	prec-min	92.6$\pm$0.3

	ifs-mix-1000-5	prec	93.5$\pm$0.1	prec-min	92.7$\pm$0.3

	ifs-mix-1000-10	prec	93.3$\pm$0.2	prec-min	92.3$\pm$0.3

	ifs-mix-1000-15	prec	94.0$\pm$0.5	prec-min	93.4$\pm$1.0

	ifs-mix-1000-20	prec	93.8$\pm$0.6	prec-min	92.0$\pm$3.3



/home/mr6564/anaconda3/envs/PyTorch/lib/python3.7/site-packages/ipykernel_launcher.py:208: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [23]:
ifsM_100_2, _, _, _, _, _ = ifs_results('domain', replay_config='ifs',\
                          ifs_option='mix', min_samples=2, memory_budget=10000)
ifsM_100_5, _, _, _, _, _ = ifs_results('domain', replay_config='ifs',\
                          ifs_option='mix', min_samples=5, memory_budget=10000)
ifsM_100_10, _, _, _, _, _ = ifs_results('domain', replay_config='ifs',\
                          ifs_option='mix', min_samples=10, memory_budget=10000)
ifsM_100_15, _, _, _, _, _ = ifs_results('domain', replay_config='ifs',\
                          ifs_option='mix', min_samples=15, memory_budget=10000)
ifsM_100_20, _, _, _, _, _ = ifs_results('domain', replay_config='ifs',\
                          ifs_option='mix', min_samples=20, memory_budget=10000)


	ifs-mix-10000-2	prec	94.3$\pm$0.3	prec-min	93.7$\pm$0.5

	ifs-mix-10000-5	prec	94.3$\pm$0.1	prec-min	93.5$\pm$0.3

	ifs-mix-10000-10	prec	94.3$\pm$0.2	prec-min	93.7$\pm$0.3

	ifs-mix-10000-15	prec	94.1$\pm$0.2	prec-min	93.3$\pm$0.2

	ifs-mix-10000-20	prec	94.0$\pm$0.2	prec-min	93.3$\pm$0.3



/home/mr6564/anaconda3/envs/PyTorch/lib/python3.7/site-packages/ipykernel_launcher.py:208: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [24]:
ifsM_100_2, _, _, _, _, _ = ifs_results('domain', replay_config='ifs',\
                          ifs_option='mix', min_samples=2, memory_budget=50000)
ifsM_100_5, _, _, _, _, _ = ifs_results('domain', replay_config='ifs',\
                          ifs_option='mix', min_samples=5, memory_budget=50000)
ifsM_100_10, _, _, _, _, _ = ifs_results('domain', replay_config='ifs',\
                          ifs_option='mix', min_samples=10, memory_budget=50000)
ifsM_100_15, _, _, _, _, _ = ifs_results('domain', replay_config='ifs',\
                          ifs_option='mix', min_samples=15, memory_budget=50000)
ifsM_100_20, _, _, _, _, _ = ifs_results('domain', replay_config='ifs',\
                          ifs_option='mix', min_samples=20, memory_budget=50000)


	ifs-mix-50000-2	prec	95.4$\pm$0.1	prec-min	94.5$\pm$0.3

	ifs-mix-50000-5	prec	95.2$\pm$0.2	prec-min	94.4$\pm$0.3

	ifs-mix-50000-10	prec	95.3$\pm$0.2	prec-min	94.6$\pm$0.3

	ifs-mix-50000-15	prec	95.3$\pm$0.1	prec-min	94.3$\pm$0.5

	ifs-mix-50000-20	prec	95.0$\pm$0.3	prec-min	94.0$\pm$0.4



/home/mr6564/anaconda3/envs/PyTorch/lib/python3.7/site-packages/ipykernel_launcher.py:208: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [42]:
ifsM_100_2, _, _, _, _, _ = ifs_results('domain', replay_config='ifs',\
                          ifs_option='mix', min_samples=2, memory_budget=100000)
ifsM_100_5, _, _, _, _, _ = ifs_results('domain', replay_config='ifs',\
                          ifs_option='mix', min_samples=5, memory_budget=100000)
ifsM_100_10, _, _, _, _, _ = ifs_results('domain', replay_config='ifs',\
                          ifs_option='mix', min_samples=10, memory_budget=100000)
ifsM_100_15, _, _, _, _, _ = ifs_results('domain', replay_config='ifs',\
                          ifs_option='mix', min_samples=15, memory_budget=100000)
ifsM_100_20, _, _, _, _, _ = ifs_results('domain', replay_config='ifs',\
                          ifs_option='mix', min_samples=20, memory_budget=100000)


	ifs-mix-100000-2	prec	95.4$\pm$0.5	prec-min	94.2$\pm$0.6

	ifs-mix-100000-5	prec	95.7$\pm$0.1	prec-min	94.5$\pm$0.3

	ifs-mix-100000-10	prec	95.8$\pm$0.0	prec-min	94.3$\pm$0.2

	ifs-mix-100000-15	prec	95.7$\pm$0.2	prec-min	94.2$\pm$0.5

	ifs-mix-100000-20	prec	95.6$\pm$0.2	prec-min	93.7$\pm$1.1



/home/mr6564/anaconda3/envs/PyTorch/lib/python3.7/site-packages/ipykernel_launcher.py:208: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [27]:
ifsM_100_2, _, _, _, _, _ = ifs_results('domain', replay_config='ifs',\
                          ifs_option='mix', min_samples=2, memory_budget=150000)
ifsM_100_5, _, _, _, _, _ = ifs_results('domain', replay_config='ifs',\
                          ifs_option='mix', min_samples=5, memory_budget=150000)
ifsM_100_10, _, _, _, _, _ = ifs_results('domain', replay_config='ifs',\
                          ifs_option='mix', min_samples=10, memory_budget=150000)
ifsM_100_15, _, _, _, _, _ = ifs_results('domain', replay_config='ifs',\
                          ifs_option='mix', min_samples=15, memory_budget=150000)
ifsM_100_20, _, _, _, _, _ = ifs_results('domain', replay_config='ifs',\
                          ifs_option='mix', min_samples=20, memory_budget=150000)


	ifs-mix-150000-2	prec	95.9$\pm$0.1	prec-min	94.5$\pm$0.4

	ifs-mix-150000-5	prec	95.9$\pm$0.1	prec-min	94.2$\pm$0.6

	ifs-mix-150000-10	prec	95.9$\pm$0.1	prec-min	94.2$\pm$0.3

	ifs-mix-150000-15	prec	95.9$\pm$0.1	prec-min	94.1$\pm$0.4

	ifs-mix-150000-20	prec	95.7$\pm$0.5	prec-min	94.0$\pm$0.6



/home/mr6564/anaconda3/envs/PyTorch/lib/python3.7/site-packages/ipykernel_launcher.py:208: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [37]:
ifsM_100_2, _, _, _, _, _ = ifs_results('domain', replay_config='ifs',\
                          ifs_option='mix', min_samples=2, memory_budget=200000)
ifsM_100_5, _, _, _, _, _ = ifs_results('domain', replay_config='ifs',\
                          ifs_option='mix', min_samples=5, memory_budget=200000)
ifsM_100_10, _, _, _, _, _ = ifs_results('domain', replay_config='ifs',\
                          ifs_option='mix', min_samples=10, memory_budget=200000)
ifsM_100_15, _, _, _, _, _ = ifs_results('domain', replay_config='ifs',\
                          ifs_option='mix', min_samples=15, memory_budget=200000)
ifsM_100_20, _, _, _, _, _ = ifs_results('domain', replay_config='ifs',\
                          ifs_option='mix', min_samples=20, memory_budget=200000)


	ifs-mix-200000-2	prec	95.9$\pm$0.1	prec-min	93.6$\pm$0.8

	ifs-mix-200000-5	prec	96.1$\pm$0.1	prec-min	94.5$\pm$0.2

	ifs-mix-200000-10	prec	95.9$\pm$0.3	prec-min	92.8$\pm$3.4

	ifs-mix-200000-15	prec	96.0$\pm$0.1	prec-min	94.1$\pm$0.4

	ifs-mix-200000-20	prec	96.0$\pm$0.1	prec-min	94.2$\pm$0.3



In [31]:
ifsM_100_2, _, _, _, _, _ = ifs_results('domain', replay_config='ifs',\
                          ifs_option='mix', min_samples=2, memory_budget=250000)
ifsM_100_5, _, _, _, _, _ = ifs_results('domain', replay_config='ifs',\
                          ifs_option='mix', min_samples=5, memory_budget=250000)
ifsM_100_10, _, _, _, _, _ = ifs_results('domain', replay_config='ifs',\
                          ifs_option='mix', min_samples=10, memory_budget=250000)
ifsM_100_15, _, _, _, _, _ = ifs_results('domain', replay_config='ifs',\
                          ifs_option='mix', min_samples=15, memory_budget=250000)
ifsM_100_20, _, _, _, _, _ = ifs_results('domain', replay_config='ifs',\
                          ifs_option='mix', min_samples=20, memory_budget=250000)


	ifs-mix-250000-2	prec	96.1$\pm$0.1	prec-min	94.5$\pm$0.4

	ifs-mix-250000-5	prec	96.0$\pm$0.1	prec-min	94.0$\pm$0.5

	ifs-mix-250000-10	prec	96.0$\pm$0.1	prec-min	94.0$\pm$0.4

	ifs-mix-250000-15	prec	96.1$\pm$0.1	prec-min	94.4$\pm$0.3

	ifs-mix-250000-20	prec	96.1$\pm$0.1	prec-min	94.5$\pm$0.3



In [38]:
ifsM_100_2, _, _, _, _, _ = ifs_results('domain', replay_config='ifs',\
                          ifs_option='mix', min_samples=2, memory_budget=300000)
ifsM_100_5, _, _, _, _, _ = ifs_results('domain', replay_config='ifs',\
                          ifs_option='mix', min_samples=5, memory_budget=300000)
ifsM_100_10, _, _, _, _, _ = ifs_results('domain', replay_config='ifs',\
                          ifs_option='mix', min_samples=10, memory_budget=300000)
ifsM_100_15, _, _, _, _, _ = ifs_results('domain', replay_config='ifs',\
                          ifs_option='mix', min_samples=15, memory_budget=300000)
ifsM_100_20, _, _, _, _, _ = ifs_results('domain', replay_config='ifs',\
                          ifs_option='mix', min_samples=20, memory_budget=300000)


	ifs-mix-300000-2	prec	96.2$\pm$0.0	prec-min	94.4$\pm$0.2

	ifs-mix-300000-5	prec	96.1$\pm$0.1	prec-min	94.3$\pm$0.3

	ifs-mix-300000-10	prec	96.2$\pm$0.1	prec-min	94.5$\pm$0.2

	ifs-mix-300000-15	prec	96.0$\pm$0.1	prec-min	93.8$\pm$0.3

	ifs-mix-300000-20	prec	96.1$\pm$0.1	prec-min	94.2$\pm$0.2



In [32]:
ifsM_100_2, _, _, _, _, _ = ifs_results('domain', replay_config='ifs',\
                          ifs_option='mix', min_samples=2, memory_budget=350000)
ifsM_100_5, _, _, _, _, _ = ifs_results('domain', replay_config='ifs',\
                          ifs_option='mix', min_samples=5, memory_budget=350000)
ifsM_100_10, _, _, _, _, _ = ifs_results('domain', replay_config='ifs',\
                          ifs_option='mix', min_samples=10, memory_budget=350000)
ifsM_100_15, _, _, _, _, _ = ifs_results('domain', replay_config='ifs',\
                          ifs_option='mix', min_samples=15, memory_budget=350000)
ifsM_100_20, _, _, _, _, _ = ifs_results('domain', replay_config='ifs',\
                          ifs_option='mix', min_samples=20, memory_budget=350000)


	ifs-mix-350000-2	prec	96.1$\pm$0.1	prec-min	94.5$\pm$0.5

	ifs-mix-350000-5	prec	96.0$\pm$0.1	prec-min	94.3$\pm$0.5

	ifs-mix-350000-10	prec	96.1$\pm$0.1	prec-min	94.0$\pm$0.5

	ifs-mix-350000-15	prec	96.1$\pm$0.2	prec-min	94.0$\pm$0.7

	ifs-mix-350000-20	prec	96.1$\pm$0.2	prec-min	93.9$\pm$1.1



In [41]:
ifsM_100_2, _, _, _, _, _ = ifs_results('domain', replay_config='ifs',\
                          ifs_option='mix', min_samples=2, memory_budget=400000)
ifsM_100_5, _, _, _, _, _ = ifs_results('domain', replay_config='ifs',\
                          ifs_option='mix', min_samples=5, memory_budget=400000)
ifsM_100_10, _, _, _, _, _ = ifs_results('domain', replay_config='ifs',\
                          ifs_option='mix', min_samples=10, memory_budget=400000)
ifsM_100_15, _, _, _, _, _ = ifs_results('domain', replay_config='ifs',\
                          ifs_option='mix', min_samples=15, memory_budget=400000)
ifsM_100_20, _, _, _, _, _ = ifs_results('domain', replay_config='ifs',\
                          ifs_option='mix', min_samples=20, memory_budget=400000)


	ifs-mix-400000-2	prec	96.1$\pm$0.1	prec-min	94.0$\pm$1.2

	ifs-mix-400000-5	prec	96.1$\pm$0.1	prec-min	94.2$\pm$0.2

	ifs-mix-400000-10	prec	96.1$\pm$0.1	prec-min	94.7$\pm$0.4

	ifs-mix-400000-15	prec	96.1$\pm$0.2	prec-min	94.1$\pm$0.9

	ifs-mix-400000-20	prec	96.2$\pm$0.1	prec-min	94.7$\pm$0.0



In [45]:
ifs_1K, _, _, _, _, _ = ifs_results('domain',  replay_config='ifs', ifs_option='ratio', goodware_ifs=True, memory_budget=1000)
ifs_10K, _, _, _, _, _ = ifs_results('domain',  replay_config='ifs', ifs_option='ratio', goodware_ifs=True, memory_budget=10000)
ifs_50K, _, _, _, _, _ = ifs_results('domain',  replay_config='ifs', ifs_option='ratio', goodware_ifs=True, memory_budget=50000)
ifs_100K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='ratio', goodware_ifs=True, memory_budget=100000)
ifs_150K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='ratio', goodware_ifs=True, memory_budget=150000)
ifs_200K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='ratio', goodware_ifs=True, memory_budget=200000)
ifs_250K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='ratio', goodware_ifs=True, memory_budget=250000)
ifs_300K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='ratio', goodware_ifs=True, memory_budget=300000)
ifs_350K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='ratio', goodware_ifs=True, memory_budget=350000)
ifs_400K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='ratio', goodware_ifs=True, memory_budget=400000);

	Good-ifs-ratio-1000	prec	93.5$\pm$0.2	prec-min	92.7$\pm$0.2

	Good-ifs-ratio-10000	prec	94.1$\pm$0.3	prec-min	93.3$\pm$0.2

	Good-ifs-ratio-50000	prec	95.0$\pm$0.1	prec-min	93.9$\pm$0.5

	Good-ifs-ratio-100000	prec	95.7$\pm$0.1	prec-min	94.3$\pm$0.4

	Good-ifs-ratio-150000	prec	96.0$\pm$0.1	prec-min	94.7$\pm$0.5

	Good-ifs-ratio-200000	prec	96.0$\pm$0.1	prec-min	94.1$\pm$0.3

	Good-ifs-ratio-250000	prec	96.1$\pm$0.1	prec-min	94.4$\pm$0.6

	Good-ifs-ratio-300000	prec	96.1$\pm$0.1	prec-min	94.2$\pm$0.5

	Good-ifs-ratio-350000	prec	96.0$\pm$0.2	prec-min	94.1$\pm$0.3

	Good-ifs-ratio-400000	prec	96.0$\pm$0.3	prec-min	93.9$\pm$0.5



/home/mr6564/anaconda3/envs/PyTorch/lib/python3.7/site-packages/ipykernel_launcher.py:222: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [50]:
ifs_1K, _, _, _, _, _ = ifs_results('domain',  replay_config='ifs', ifs_option='uniform', goodware_ifs=True, memory_budget=1000)
ifs_10K, _, _, _, _, _ = ifs_results('domain',  replay_config='ifs', ifs_option='uniform', goodware_ifs=True, memory_budget=10000)
ifs_50K, _, _, _, _, _ = ifs_results('domain',  replay_config='ifs', ifs_option='uniform', goodware_ifs=True, memory_budget=50000)
ifs_100K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='uniform', goodware_ifs=True, memory_budget=100000)
ifs_150K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='uniform', goodware_ifs=True, memory_budget=150000)
ifs_200K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='uniform', goodware_ifs=True, memory_budget=200000)
ifs_250K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='uniform', goodware_ifs=True, memory_budget=250000)
ifs_300K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='uniform', goodware_ifs=True, memory_budget=300000)
ifs_350K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='uniform', goodware_ifs=True, memory_budget=350000)
ifs_400K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='uniform', goodware_ifs=True, memory_budget=400000);


	Good-ifs-uniform-1000	prec	93.6$\pm$0.2	prec-min	92.6$\pm$0.2

	Good-ifs-uniform-10000	prec	94.1$\pm$0.1	prec-min	93.0$\pm$0.3

	Good-ifs-uniform-50000	prec	94.8$\pm$0.1	prec-min	94.2$\pm$0.2

	Good-ifs-uniform-100000	prec	95.1$\pm$0.1	prec-min	94.1$\pm$0.4

	Good-ifs-uniform-150000	prec	95.3$\pm$0.1	prec-min	94.1$\pm$0.6

	Good-ifs-uniform-200000	prec	95.6$\pm$0.1	prec-min	94.4$\pm$0.1

	Good-ifs-uniform-250000	prec	95.5$\pm$0.1	prec-min	94.3$\pm$0.3

	Good-ifs-uniform-300000	prec	95.5$\pm$0.1	prec-min	94.2$\pm$0.2

	Good-ifs-uniform-350000	prec	95.6$\pm$0.1	prec-min	94.3$\pm$0.4

	Good-ifs-uniform-400000	prec	95.5$\pm$0.0	prec-min	93.0$\pm$0.2



In [52]:
ifs_1K, _, _, _, _, _ = ifs_results('domain',  replay_config='ifs', ifs_option='mix', min_samples=2, goodware_ifs=True, memory_budget=1000)
ifs_10K, _, _, _, _, _ = ifs_results('domain',  replay_config='ifs', ifs_option='mix', min_samples=2, goodware_ifs=True, memory_budget=10000)
ifs_50K, _, _, _, _, _ = ifs_results('domain',  replay_config='ifs', ifs_option='mix', min_samples=2, goodware_ifs=True, memory_budget=50000)
ifs_100K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='mix', min_samples=2, goodware_ifs=True, memory_budget=100000)
ifs_150K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='mix', min_samples=2, goodware_ifs=True, memory_budget=150000)
ifs_200K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='mix', min_samples=2, goodware_ifs=True, memory_budget=200000)
ifs_250K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='mix', min_samples=2, goodware_ifs=True, memory_budget=250000)
ifs_300K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='mix', min_samples=2, goodware_ifs=True, memory_budget=300000)
ifs_350K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='mix', min_samples=2, goodware_ifs=True, memory_budget=350000)
ifs_400K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='mix', min_samples=2, goodware_ifs=True, memory_budget=400000);


	Good-ifs-mix-1000-2	prec	93.9$\pm$0.3	prec-min	93.0$\pm$0.5

	Good-ifs-mix-10000-2	prec	94.3$\pm$0.2	prec-min	93.6$\pm$0.3

	Good-ifs-mix-50000-2	prec	95.1$\pm$0.1	prec-min	94.4$\pm$0.5

	Good-ifs-mix-100000-2	prec	95.7$\pm$0.0	prec-min	94.3$\pm$0.2

	Good-ifs-mix-150000-2	prec	95.9$\pm$0.0	prec-min	94.4$\pm$0.4

	Good-ifs-mix-200000-2	prec	96.0$\pm$0.1	prec-min	94.4$\pm$0.4

	Good-ifs-mix-250000-2	prec	96.0$\pm$0.0	prec-min	94.2$\pm$0.4

	Good-ifs-mix-300000-2	prec	96.2$\pm$0.0	prec-min	94.4$\pm$0.4

	Good-ifs-mix-350000-2	prec	95.7$\pm$0.6	prec-min	89.5$\pm$7.8

	Good-ifs-mix-400000-2	prec	96.2$\pm$0.1	prec-min	94.2$\pm$0.5



/home/mr6564/anaconda3/envs/PyTorch/lib/python3.7/site-packages/ipykernel_launcher.py:222: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [54]:
ifs_1K, _, _, _, _, _ = ifs_results('domain',  replay_config='ifs', ifs_option='mix', min_samples=5, goodware_ifs=True, memory_budget=1000)
ifs_10K, _, _, _, _, _ = ifs_results('domain',  replay_config='ifs', ifs_option='mix', min_samples=5, goodware_ifs=True, memory_budget=10000)
ifs_50K, _, _, _, _, _ = ifs_results('domain',  replay_config='ifs', ifs_option='mix', min_samples=5, goodware_ifs=True, memory_budget=50000)
ifs_100K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='mix', min_samples=5, goodware_ifs=True, memory_budget=100000)
ifs_150K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='mix', min_samples=5, goodware_ifs=True, memory_budget=150000)
ifs_200K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='mix', min_samples=5, goodware_ifs=True, memory_budget=200000)
ifs_250K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='mix', min_samples=5, goodware_ifs=True, memory_budget=250000)
ifs_300K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='mix', min_samples=5, goodware_ifs=True, memory_budget=300000)
ifs_350K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='mix', min_samples=5, goodware_ifs=True, memory_budget=350000)
ifs_400K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='mix', min_samples=5, goodware_ifs=True, memory_budget=400000);


	Good-ifs-mix-1000-5	prec	93.2$\pm$0.3	prec-min	92.2$\pm$0.6

	Good-ifs-mix-10000-5	prec	94.1$\pm$0.1	prec-min	93.3$\pm$0.1

	Good-ifs-mix-50000-5	prec	95.0$\pm$0.1	prec-min	94.1$\pm$0.4

	Good-ifs-mix-100000-5	prec	95.7$\pm$0.1	prec-min	94.4$\pm$0.3

	Good-ifs-mix-150000-5	prec	95.9$\pm$0.1	prec-min	94.5$\pm$0.3

	Good-ifs-mix-200000-5	prec	96.0$\pm$0.1	prec-min	94.6$\pm$0.0

	Good-ifs-mix-250000-5	prec	96.0$\pm$0.1	prec-min	94.3$\pm$0.2

	Good-ifs-mix-300000-5	prec	96.2$\pm$0.1	prec-min	94.5$\pm$0.5

	Good-ifs-mix-350000-5	prec	96.2$\pm$0.0	prec-min	94.7$\pm$0.1

	Good-ifs-mix-400000-5	prec	96.1$\pm$0.1	prec-min	94.2$\pm$0.4



In [56]:
ifs_1K, _, _, _, _, _ = ifs_results('domain',  replay_config='ifs', ifs_option='mix', min_samples=10, goodware_ifs=True, memory_budget=1000)
ifs_10K, _, _, _, _, _ = ifs_results('domain',  replay_config='ifs', ifs_option='mix', min_samples=10, goodware_ifs=True, memory_budget=10000)
ifs_50K, _, _, _, _, _ = ifs_results('domain',  replay_config='ifs', ifs_option='mix', min_samples=10, goodware_ifs=True, memory_budget=50000)
ifs_100K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='mix', min_samples=10, goodware_ifs=True, memory_budget=100000)
ifs_150K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='mix', min_samples=10, goodware_ifs=True, memory_budget=150000)
ifs_200K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='mix', min_samples=10, goodware_ifs=True, memory_budget=200000)
ifs_250K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='mix', min_samples=10, goodware_ifs=True, memory_budget=250000)
ifs_300K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='mix', min_samples=10, goodware_ifs=True, memory_budget=300000)
ifs_350K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='mix', min_samples=10, goodware_ifs=True, memory_budget=350000)
ifs_400K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='mix', min_samples=10, goodware_ifs=True, memory_budget=400000);


	Good-ifs-mix-1000-10	prec	93.4$\pm$0.1	prec-min	92.3$\pm$0.3

	Good-ifs-mix-10000-10	prec	94.2$\pm$0.2	prec-min	93.3$\pm$0.3

	Good-ifs-mix-50000-10	prec	95.1$\pm$0.0	prec-min	94.4$\pm$0.4

	Good-ifs-mix-100000-10	prec	95.7$\pm$0.1	prec-min	94.4$\pm$0.5

	Good-ifs-mix-150000-10	prec	95.9$\pm$0.1	prec-min	94.1$\pm$0.8

	Good-ifs-mix-200000-10	prec	95.9$\pm$0.1	prec-min	94.1$\pm$0.3

	Good-ifs-mix-250000-10	prec	96.1$\pm$0.1	prec-min	94.9$\pm$0.4

	Good-ifs-mix-300000-10	prec	96.1$\pm$0.1	prec-min	94.1$\pm$0.3

	Good-ifs-mix-350000-10	prec	96.2$\pm$0.0	prec-min	94.5$\pm$0.4

	Good-ifs-mix-400000-10	prec	96.1$\pm$0.0	prec-min	94.4$\pm$0.5



/home/mr6564/anaconda3/envs/PyTorch/lib/python3.7/site-packages/ipykernel_launcher.py:222: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [58]:
ifs_1K, _, _, _, _, _ = ifs_results('domain',  replay_config='ifs', ifs_option='mix', min_samples=15, goodware_ifs=True, memory_budget=1000)
ifs_10K, _, _, _, _, _ = ifs_results('domain',  replay_config='ifs', ifs_option='mix', min_samples=15, goodware_ifs=True, memory_budget=10000)
ifs_50K, _, _, _, _, _ = ifs_results('domain',  replay_config='ifs', ifs_option='mix', min_samples=15, goodware_ifs=True, memory_budget=50000)
ifs_100K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='mix', min_samples=15, goodware_ifs=True, memory_budget=100000)
ifs_150K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='mix', min_samples=15, goodware_ifs=True, memory_budget=150000)
ifs_200K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='mix', min_samples=15, goodware_ifs=True, memory_budget=200000)
ifs_250K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='mix', min_samples=15, goodware_ifs=True, memory_budget=250000)
ifs_300K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='mix', min_samples=15, goodware_ifs=True, memory_budget=300000)
ifs_350K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='mix', min_samples=15, goodware_ifs=True, memory_budget=350000)
ifs_400K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='mix', min_samples=15, goodware_ifs=True, memory_budget=400000);


	Good-ifs-mix-1000-15	prec	93.6$\pm$0.4	prec-min	92.4$\pm$0.5

	Good-ifs-mix-10000-15	prec	94.1$\pm$0.1	prec-min	93.1$\pm$0.0

	Good-ifs-mix-50000-15	prec	95.0$\pm$0.2	prec-min	94.0$\pm$0.4

	Good-ifs-mix-100000-15	prec	95.7$\pm$0.1	prec-min	94.4$\pm$0.5

	Good-ifs-mix-150000-15	prec	95.9$\pm$0.0	prec-min	94.6$\pm$0.3

	Good-ifs-mix-200000-15	prec	96.0$\pm$0.1	prec-min	94.7$\pm$0.3

	Good-ifs-mix-250000-15	prec	96.1$\pm$0.1	prec-min	94.3$\pm$0.7

	Good-ifs-mix-300000-15	prec	96.1$\pm$0.1	prec-min	94.3$\pm$0.4

	Good-ifs-mix-350000-15	prec	96.1$\pm$0.1	prec-min	94.4$\pm$0.3

	Good-ifs-mix-400000-15	prec	96.2$\pm$0.1	prec-min	94.2$\pm$0.6



In [59]:
ifs_1K, _, _, _, _, _ = ifs_results('domain',  replay_config='ifs', ifs_option='mix', min_samples=20, goodware_ifs=True, memory_budget=1000)
ifs_10K, _, _, _, _, _ = ifs_results('domain',  replay_config='ifs', ifs_option='mix', min_samples=20, goodware_ifs=True, memory_budget=10000)
ifs_50K, _, _, _, _, _ = ifs_results('domain',  replay_config='ifs', ifs_option='mix', min_samples=20, goodware_ifs=True, memory_budget=50000)
ifs_100K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='mix', min_samples=20, goodware_ifs=True, memory_budget=100000)
ifs_150K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='mix', min_samples=20, goodware_ifs=True, memory_budget=150000)
ifs_200K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='mix', min_samples=20, goodware_ifs=True, memory_budget=200000)
ifs_250K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='mix', min_samples=20, goodware_ifs=True, memory_budget=250000)
ifs_300K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='mix', min_samples=20, goodware_ifs=True, memory_budget=300000)
ifs_350K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='mix', min_samples=20, goodware_ifs=True, memory_budget=350000)
ifs_400K, _, _, _, _, _ = ifs_results('domain', replay_config='ifs', ifs_option='mix', min_samples=20, goodware_ifs=True, memory_budget=400000);


	Good-ifs-mix-1000-20	prec	93.7$\pm$0.2	prec-min	92.7$\pm$0.2

	Good-ifs-mix-10000-20	prec	94.0$\pm$0.2	prec-min	92.9$\pm$0.3

	Good-ifs-mix-50000-20	prec	95.0$\pm$0.2	prec-min	93.9$\pm$0.2

	Good-ifs-mix-100000-20	prec	95.7$\pm$0.1	prec-min	94.7$\pm$0.4

	Good-ifs-mix-150000-20	prec	95.9$\pm$0.2	prec-min	94.2$\pm$0.3

	Good-ifs-mix-200000-20	prec	96.0$\pm$0.0	prec-min	94.3$\pm$0.4

	Good-ifs-mix-250000-20	prec	96.0$\pm$0.1	prec-min	94.5$\pm$0.3

	Good-ifs-mix-300000-20	prec	96.2$\pm$0.1	prec-min	94.4$\pm$0.3

	Good-ifs-mix-350000-20	prec	96.2$\pm$0.1	prec-min	94.2$\pm$0.7

	Good-ifs-mix-400000-20	prec	96.2$\pm$0.1	prec-min	94.4$\pm$0.5



In [67]:
ifs_1K, _, _, _, _, _ = ifs_results('domain',  replay_config='aws', ifs_option='ratio', memory_budget=1000)
ifs_10K, _, _, _, _, _ = ifs_results('domain',  replay_config='aws', ifs_option='ratio', memory_budget=10000)
ifs_50K, _, _, _, _, _ = ifs_results('domain',  replay_config='aws', ifs_option='ratio', memory_budget=50000)
ifs_100K, _, _, _, _, _ = ifs_results('domain', replay_config='aws', ifs_option='ratio', memory_budget=100000)
ifs_150K, _, _, _, _, _ = ifs_results('domain', replay_config='aws', ifs_option='ratio', memory_budget=150000)
ifs_200K, _, _, _, _, _ = ifs_results('domain', replay_config='aws', ifs_option='ratio', memory_budget=200000)
ifs_250K, _, _, _, _, _ = ifs_results('domain', replay_config='aws', ifs_option='ratio', memory_budget=250000)
ifs_300K, _, _, _, _, _ = ifs_results('domain', replay_config='aws', ifs_option='ratio', memory_budget=300000)
ifs_350K, _, _, _, _, _ = ifs_results('domain', replay_config='aws', ifs_option='ratio', memory_budget=350000)
ifs_400K, _, _, _, _, _ = ifs_results('domain', replay_config='aws', ifs_option='ratio', memory_budget=400000);

	aws-ratio-1000	prec	93.6$\pm$0.1	prec-min	92.5$\pm$0.4

	aws-ratio-10000	prec	94.4$\pm$0.3	prec-min	93.7$\pm$0.4

	aws-ratio-50000	prec	95.3$\pm$0.2	prec-min	94.3$\pm$0.2

	aws-ratio-100000	prec	95.8$\pm$0.1	prec-min	94.3$\pm$0.5

	aws-ratio-150000	prec	95.9$\pm$0.1	prec-min	94.2$\pm$0.3

	aws-ratio-200000	prec	96.1$\pm$0.1	prec-min	94.9$\pm$0.3

	aws-ratio-250000	prec	96.1$\pm$0.1	prec-min	94.6$\pm$0.1

	aws-ratio-300000	prec	96.1$\pm$0.1	prec-min	94.2$\pm$0.5

	aws-ratio-350000	prec	96.2$\pm$0.1	prec-min	94.7$\pm$0.4

	aws-ratio-400000	prec	96.1$\pm$0.1	prec-min	94.1$\pm$0.4



In [68]:
ifs_1K, _, _, _, _, _ = ifs_results('domain',  replay_config='aws', ifs_option='uniform', memory_budget=1000)
ifs_10K, _, _, _, _, _ = ifs_results('domain',  replay_config='aws', ifs_option='uniform', memory_budget=10000)
ifs_50K, _, _, _, _, _ = ifs_results('domain',  replay_config='aws', ifs_option='uniform', memory_budget=50000)
ifs_100K, _, _, _, _, _ = ifs_results('domain', replay_config='aws', ifs_option='uniform', memory_budget=100000)
ifs_150K, _, _, _, _, _ = ifs_results('domain', replay_config='aws', ifs_option='uniform', memory_budget=150000)
ifs_200K, _, _, _, _, _ = ifs_results('domain', replay_config='aws', ifs_option='uniform', memory_budget=200000)
ifs_250K, _, _, _, _, _ = ifs_results('domain', replay_config='aws', ifs_option='uniform', memory_budget=250000)
ifs_300K, _, _, _, _, _ = ifs_results('domain', replay_config='aws', ifs_option='uniform', memory_budget=300000)
ifs_350K, _, _, _, _, _ = ifs_results('domain', replay_config='aws', ifs_option='uniform', memory_budget=350000)
ifs_400K, _, _, _, _, _ = ifs_results('domain', replay_config='aws', ifs_option='uniform', memory_budget=400000);

	aws-uniform-1000	prec	93.1$\pm$0.5	prec-min	91.9$\pm$0.9

	aws-uniform-10000	prec	94.1$\pm$0.2	prec-min	93.1$\pm$0.3

	aws-uniform-50000	prec	94.9$\pm$0.1	prec-min	93.9$\pm$0.3

	aws-uniform-100000	prec	95.2$\pm$0.2	prec-min	94.0$\pm$0.5

	aws-uniform-150000	prec	95.5$\pm$0.1	prec-min	94.4$\pm$0.3

	aws-uniform-200000	prec	95.6$\pm$0.1	prec-min	94.2$\pm$0.3

	aws-uniform-250000	prec	95.6$\pm$0.0	prec-min	94.2$\pm$0.5

	aws-uniform-300000	prec	95.7$\pm$0.1	prec-min	93.9$\pm$0.5

	aws-uniform-350000	prec	95.6$\pm$0.1	prec-min	94.2$\pm$0.5

	aws-uniform-400000	prec	95.7$\pm$0.1	prec-min	94.2$\pm$0.3

